# Анализ структуры баз данных проекта

Этот ноутбук подключается к базам данных `cms`, `lists` и `filestorage` и выводит информацию о структуре таблиц (колонки, типы данных, ограничения).

In [1]:
import os
import sys
from pathlib import Path
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown

In [6]:
# Устанавливаем опции Pandas для полного отображения
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


In [2]:
# Загружаем переменные окружения из .env в корне проекта
# Путь к корню проекта (предполагается, что ноутбук в notebooks/)
project_root = Path.cwd().parent
dotenv_path = project_root / '.env'

if not dotenv_path.exists():
    print(f"Ошибка: Файл .env не найден по пути {dotenv_path}", file=sys.stderr)
else:
    load_dotenv(dotenv_path=dotenv_path)
    print(f".env файл загружен из {dotenv_path}")

# Получаем параметры подключения
pg_user = os.getenv("PG_USER")
pg_password = os.getenv("PG_PASSWORD")
pg_host = os.getenv("PG_HOST", "localhost")
pg_port = os.getenv("PG_PORT", "5432")

if not all([pg_user, pg_password]):
    print("Ошибка: Переменные PG_USER и PG_PASSWORD должны быть установлены в .env.", file=sys.stderr)

.env файл загружен из /home/skitarii/PycharmProjects/chatHRD/.env


## Вспомогательные функции

In [4]:
def get_connection(db_name: str):
    """Подключается к указанной БД."""
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=pg_user,
            password=pg_password,
            host=pg_host,
            port=pg_port
        )
        print(f"Успешное подключение к БД '{db_name}'.")
    except psycopg2.OperationalError as e:
        print(f"Ошибка подключения к БД '{db_name}': {e}", file=sys.stderr)
    return conn

def get_tables(conn) -> list[str]:
    """Возвращает список пользовательских таблиц в текущей схеме."""
    if not conn:
        return []
    query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
        ORDER BY table_name;
    """
    try:
        df = pd.read_sql_query(query, conn)
        return df['table_name'].tolist()
    except Exception as e:
        print(f"Ошибка получения списка таблиц: {e}", file=sys.stderr)
        return []

def get_table_schema(conn, table_name: str) -> pd.DataFrame:
    """Получает схему таблицы (колонки, типы, nullability)."""
    if not conn:
        return pd.DataFrame()
    query = f"""
        SELECT
            column_name,
            data_type,
            is_nullable,
            column_default
        FROM information_schema.columns
        WHERE table_schema = 'public' AND table_name = '{table_name}'
        ORDER BY ordinal_position;
    """
    try:
        df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        print(f"Ошибка получения схемы для таблицы {table_name}: {e}", file=sys.stderr)
        return pd.DataFrame()

def get_table_constraints(conn, table_name: str) -> pd.DataFrame:
    """Получает информацию об ограничениях (PK, FK) таблицы."""
    if not conn:
        return pd.DataFrame()
    # Этот запрос немного упрощен, для более сложных случаев может потребоваться доработка
    query = f"""
        SELECT
            tc.constraint_name,
            tc.constraint_type,
            kcu.column_name,
            ccu.table_name AS foreign_table_name,
            ccu.column_name AS foreign_column_name
        FROM
            information_schema.table_constraints AS tc
            JOIN information_schema.key_column_usage AS kcu
              ON tc.constraint_name = kcu.constraint_name
              AND tc.table_schema = kcu.table_schema
            LEFT JOIN information_schema.constraint_column_usage AS ccu
              ON ccu.constraint_name = tc.constraint_name
              AND ccu.table_schema = tc.table_schema
        WHERE tc.table_schema = 'public' AND tc.table_name='{table_name}'
        ORDER BY tc.constraint_type, tc.constraint_name, kcu.ordinal_position;
    """
    try:
        df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        print(f"Ошибка получения ограничений для таблицы {table_name}: {e}", file=sys.stderr)
        return pd.DataFrame()

## Анализ баз данных

In [5]:
databases_to_analyze = ["cms", "lists", "filestorage"]


for db_name in databases_to_analyze:
    display(Markdown(f'\n## Анализ базы данных: `{db_name}`'))
    conn = get_connection(db_name)

    if conn:
        tables = get_tables(conn)
        display(Markdown(f'\n### Найдено таблиц: {len(tables)}'))

        for table_name in tables:
            display(Markdown(f'\n#### Таблица: `{table_name}`'))

            display(Markdown('\n##### Схема:'))
            schema_df = get_table_schema(conn, table_name)
            if not schema_df.empty:
                display(schema_df)
            else:
                print(f"Не удалось получить схему для {table_name}")

            display(Markdown('\n##### Ограничения (PK, FK):'))
            constraints_df = get_table_constraints(conn, table_name)
            if not constraints_df.empty:
                display(constraints_df)
            else:
                 # Не выводим сообщение, т.к. у таблицы может и не быть явных PK/FK
                 pass

        # Закрываем соединение после анализа базы
        conn.close()
        print(f"Соединение с БД '{db_name}' закрыто.")
    else:
        display(Markdown(f'**Не удалось подключиться к БД `{db_name}`**'))

print("\nАнализ завершен.")


## Анализ базы данных: `cms`

Успешное подключение к БД 'cms'.


/tmp/ipykernel_450849/82215137.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)



### Найдено таблиц: 34


#### Таблица: `admin_panel_users`

...

,column_name,data_type,is_nullable,column_default
0,id,integer,NO,nextval('tastypie_apikey_id_seq'::regclass)
1,key,character varying,NO,None
2,created,timestamp with time zone,NO,None
3,user_id,integer,NO,None



##### Ограничения (PK, FK):

/tmp/ipykernel_450849/82215137.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,constraint_name,constraint_type,column_name,foreign_table_name,foreign_column_name
0,tastypie_apikey_user_id_8c8fa920_fk_auth_user_id,FOREIGN KEY,user_id,auth_user,id
1,tastypie_apikey_pkey,PRIMARY KEY,id,tastypie_apikey,id
2,tastypie_apikey_user_id_key,UNIQUE,user_id,tastypie_apikey,user_id


Соединение с БД 'filestorage' закрыто.

Анализ завершен.
